In [1]:
import tensorflow as tf
import numpy as np
import math

SEPCTURAL_SAMPLES = 10
FEATURE_DIM = SEPCTURAL_SAMPLES*6*2  #120
CONV_LEN = 3
CONV_LEN_INTE = 3#4
CONV_LEN_LAST = 3#5
CONV_NUM = 64
CONV_MERGE_LEN = 3  # original 8
CONV_MERGE_LEN2 = 3  # original 6
CONV_MERGE_LEN3 = 4
CONV_NUM2 = 64
INTER_DIM = 120
OUT_DIM = 6#len(idDict)
WIDE = 20
CONV_KEEP_PROB = 0.5

BATCH_SIZE = 64
TOTAL_ITER_NUM = 1000

select = 'a'

metaDict = {'a':[119080, 1193], 'b':[116870, 1413], 'c':[116020, 1477]}
TRAIN_SIZE = metaDict[select][0]
EVAL_DATA_SIZE = metaDict[select][1]
EVAL_ITER_NUM = int(math.ceil(EVAL_DATA_SIZE / BATCH_SIZE))


In [2]:
###### Import training data
train_x = np.load('DeepSense_data/train_x.npy')
train_y = np.load('DeepSense_data/train_y.npy')
eval_x = np.load('DeepSense_data/eval_x.npy')
eval_y = np.load('DeepSense_data/eval_y.npy')

# train_x = np.array(train_x)
print('train_x.shape,train_y.shape ',train_x.shape,train_y.shape)
print('eval_x.shape,eval_y.shape ',eval_x.shape,eval_y.shape)
train_x = np.expand_dims(train_x, axis=3)
print('train_x.shape,train_y.shape ',train_x.shape,train_y.shape)
acc,gyo = np.split(train_x,2,axis=2)
print('acc.shape,gyo.shape ',acc.shape,gyo.shape)
# acc = acc.reshape((-1,20,60,1))
# gyo = gyo.reshape((-1,20,60,1))
# print(acc.shape,gyo.shape)
print(train_y[0])

train_x.shape,train_y.shape  (119080, 20, 120) (119080, 6)
eval_x.shape,eval_y.shape  (1193, 20, 120) (1193, 6)
train_x.shape,train_y.shape  (119080, 20, 120, 1) (119080, 6)
acc.shape,gyo.shape  (119080, 20, 60, 1) (119080, 20, 60, 1)
[0. 0. 0. 0. 1. 0.]


In [8]:
###### Import training data
train_x = np.load('fre_data.npy')
print(train_x.shape)
train_y = np.load('fre_data_label.npy')
# eval_x = np.load('DeepSense_data/eval_x.npy')
# eval_y = np.load('DeepSense_data/eval_y.npy')
train_x = train_x.reshape((-1,20,120))
train_x = np.expand_dims(train_x, axis=3)
# train_x = np.array(train_x)
print(train_x.shape,train_y.shape)
# print(eval_x.shape,eval_y.shape)

# acc,gyo = np.split(train_x,2)
# acc = acc.reshape((-1,20,60,1))
# gyo = gyo.reshape((-1,20,60,1))
# print(acc.shape,gyo.shape)
print(train_y[0])

(254000, 120)
(12700, 20, 120, 1) (12700, 6)
[0 0 0 0 1 0]


In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [3]:
#import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.core import Activation
from keras.layers import concatenate
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import Conv3D
from keras.layers.normalization import BatchNormalization
from keras.layers import Input
from keras import optimizers
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Reshape
import keras.backend as K
from keras.layers.recurrent import GRU,LSTM
from keras.layers import GRUCell,Lambda
from keras.layers import RNN
from keras import regularizers
# from keras.backend import concatenate

Using TensorFlow backend.


In [4]:
acc_inputs, gyro_inputs = acc,gyo
print('acc_inputs.shape,gyro_inputs.shape ',acc_inputs.shape,gyro_inputs.shape)

input1 = Input(shape=(20,60,1), name='input1')
# sensor_inputs shape (BATCH_SIZE, WIDE, FEATURE_DIM, CHANNEL=1)
acc_conv1 = Conv2D(CONV_NUM, kernel_size=[1, 2*3*CONV_LEN],
        strides=(1, 2*3), padding='VALID', activation=None, data_format='channels_last')(input1)
acc_conv1 = BatchNormalization()(acc_conv1)
acc_conv1 = Activation('relu')(acc_conv1)
acc_conv1_shape = acc_conv1.get_shape().as_list()
acc_conv1 = Dropout(CONV_KEEP_PROB, noise_shape=[acc_conv1_shape[0], 1, 1, acc_conv1_shape[3]], seed=None)(acc_conv1)
#         layers.dropout(acc_conv1, CONV_KEEP_PROB, is_training=train,
#             noise_shape=[acc_conv1_shape[0], 1, 1, acc_conv1_shape[3]], scope='acc_dropout1')

acc_conv2 = Conv2D(CONV_NUM, kernel_size=[1, CONV_LEN_INTE],
        strides=(1, 1), padding='VALID', activation=None, data_format='channels_last')(acc_conv1)
acc_conv2 = BatchNormalization()(acc_conv2)
acc_conv2 = Activation('relu')(acc_conv2)
acc_conv2_shape = acc_conv2.get_shape().as_list()
acc_conv2 = Dropout(CONV_KEEP_PROB, noise_shape=[acc_conv2_shape[0], 1, 1, acc_conv2_shape[3]], seed=None)(acc_conv2)
#layers.dropout(acc_conv2, CONV_KEEP_PROB, is_training=train,
#             noise_shape=[acc_conv2_shape[0], 1, 1, acc_conv2_shape[3]], scope='acc_dropout2')

acc_conv3 = Conv2D(CONV_NUM, kernel_size=[1, CONV_LEN_LAST],
        strides=(1, 1), padding='VALID', activation=None, data_format='channels_last')(acc_conv2)
acc_conv3 = BatchNormalization()(acc_conv3)
acc_conv3 = Activation('relu')(acc_conv3)
acc_conv3_shape = acc_conv3.get_shape().as_list()
print('acc_conv3_shape',acc_conv3_shape)
acc_conv_out = Reshape((acc_conv3_shape[1], 1, acc_conv3_shape[2],acc_conv3_shape[3]))(acc_conv3)
# tf.reshape(acc_conv3, [acc_conv3_shape[0], acc_conv3_shape[1], 1, acc_conv3_shape[2],acc_conv3_shape[3]])
print('acc_conv_out.shape ',acc_conv_out.shape)

# acc_conv_out = Reshape((-1,15*54*64))(acc_conv3)


input2 = Input(shape=(20,60,1), name='input2')
gyro_conv1 = Conv2D(CONV_NUM, kernel_size=[1, 2*3*CONV_LEN],
        strides=(1, 2*3), padding='VALID', activation=None, data_format='channels_last')(input2)
gyro_conv1 = BatchNormalization()(gyro_conv1)
gyro_conv1 = Activation('relu')(gyro_conv1)
gyro_conv1_shape = gyro_conv1.get_shape().as_list()
gyro_conv1 = Dropout(CONV_KEEP_PROB, noise_shape=[gyro_conv1_shape[0], 1, 1, gyro_conv1_shape[3]], seed=None)(gyro_conv1)

gyro_conv2 = Conv2D(CONV_NUM, kernel_size=[1, CONV_LEN_INTE],
        strides=(1, 1), padding='VALID', activation=None, data_format='channels_last')(gyro_conv1)
gyro_conv2 = BatchNormalization()(gyro_conv2)
gyro_conv2 = Activation('relu')(gyro_conv2)
gyro_conv2_shape = gyro_conv2.get_shape().as_list()
gyro_conv2 = Dropout(CONV_KEEP_PROB, noise_shape=[gyro_conv2_shape[0], 1, 1, gyro_conv2_shape[3]], seed=None)(gyro_conv2)

gyro_conv3 = Conv2D(CONV_NUM, kernel_size=[1, CONV_LEN_LAST],
        strides=(1, 1), padding='VALID', activation=None, data_format='channels_last')(gyro_conv2)
gyro_conv3 = BatchNormalization()(gyro_conv3)
gyro_conv3 = Activation('relu')(gyro_conv3)
gyro_conv3_shape = gyro_conv3.get_shape().as_list()
print('gyro_conv3.shape ',gyro_conv3_shape)
#gyro_conv_out = tf.reshape(gyro_conv3, [gyro_conv3_shape[0], gyro_conv3_shape[1], 1, gyro_conv3_shape[2], gyro_conv3_shape[3]])
gyro_conv_out = Reshape((gyro_conv3_shape[1], 1, acc_conv3_shape[2],acc_conv3_shape[3]))(gyro_conv3)
print('gyro_conv_out.shape ',gyro_conv_out.shape)

# gyro_conv_out = Reshape((-1,15*54*64))(gyro_conv3)
#print('gyro_conv_out.shape ',gyro_conv_out.shape)
sensor_conv_in = concatenate([acc_conv_out, gyro_conv_out], axis=2)
print('sensor_conv_in ',sensor_conv_in.shape)
senor_conv_shape = sensor_conv_in.get_shape().as_list()
sensor_conv_in = Dropout(CONV_KEEP_PROB,seed=None)(sensor_conv_in)
print('sensor_conv_in ',sensor_conv_in)


sensor_conv1 = Conv3D(CONV_NUM2, kernel_size=[1, 2, CONV_MERGE_LEN],
                strides=(1, 1, 1), padding='SAME', activation=None, data_format='channels_last')(sensor_conv_in)
sensor_conv1 = BatchNormalization()(sensor_conv1)
sensor_conv1 = Activation('relu')(sensor_conv1)
sensor_conv1_shape = sensor_conv1.get_shape().as_list()
sensor_conv1 = Dropout(CONV_KEEP_PROB,seed=None)(sensor_conv1)

sensor_conv2 = Conv3D(CONV_NUM2, kernel_size=[1, 2, CONV_MERGE_LEN2],
                strides=(1, 1,1), padding='SAME', activation=None, data_format='channels_last')(sensor_conv1)
sensor_conv2 = BatchNormalization()(sensor_conv2)
sensor_conv2 = Activation('relu')(sensor_conv2)
sensor_conv2_shape = sensor_conv2.get_shape().as_list()
sensor_conv2 = Dropout(CONV_KEEP_PROB,seed=None)(sensor_conv2)

sensor_conv3 = Conv3D(CONV_NUM2, kernel_size=[1, 2, CONV_MERGE_LEN3],
                strides=(1, 1,1), padding='SAME', activation=None, data_format='channels_last')(sensor_conv2)
sensor_conv3 = BatchNormalization()(sensor_conv3)
sensor_conv3 = Activation('relu')(sensor_conv3)
sensor_conv3_shape = sensor_conv3.get_shape().as_list()
print('sensor_conv3.shape ',sensor_conv3_shape)
#sensor_conv_out = Reshape(([sensor_conv3_shape[1], sensor_conv3_shape[2]*sensor_conv3_shape[3]*sensor_conv3_shape[4]]))(sensor_conv3)
#  tf.reshape(sensor_conv3, [sensor_conv3_shape[0], sensor_conv3_shape[1], sensor_conv3_shape[2]*sensor_conv3_shape[3]*sensor_conv3_shape[4]])
print('sensor_conv_out.shape ',sensor_conv3.shape)
# outputs = Flatten()(sensor_conv_out)

outputs = Reshape((20,-1))(sensor_conv3)
print('shape before gru ',outputs.shape)

# outputs=  Reshape((20,8*64))(sensor_conv_in)


outputs = GRU(INTER_DIM, activation='tanh',dropout=0.5,return_sequences=True)(outputs)
#LSTM(INTER_DIM, return_sequences=True, kernel_initializer='orthogonal', name='LSTM_1')(sensor_conv_out)     #128
outputs = Dropout(0.5)(outputs)
print('gru1.shape ',outputs.shape)

def reduce_dim(x):
    return x[:,-1:,:]

# outputs = Lambda(reduce_dim)(outputs)

# print('gru2_input',outputs[:,-4:,:].shape)
outputs = GRU(INTER_DIM, activation='tanh',dropout=0.5,return_sequences=True)(outputs)
outputs = Dropout(0.5)(outputs)
print('gru2.shape ',outputs.shape)

def reduce_timestep(x):
    x = K.mean(x, axis=1, keepdims=False)
    return x
outputs = Lambda(reduce_timestep)(outputs)
print('output of GRU ',outputs.shape)

# outputs = Flatten()(outputs)

outputs = Dense(6, activation='softmax',kernel_regularizer=regularizers.l2(5e-3))(outputs)


adam = keras.optimizers.Adam(lr=0.001, beta_1=0.5, beta_2=0.9, epsilon=None, decay=0.0, amsgrad=False)

model = Model(inputs=[input1,input2], outputs=outputs)
model.compile(loss='categorical_crossentropy',
      optimizer=adam,
      metrics=['accuracy'])

#model.fit([acc_inputs,gyro_inputs], train_y, batch_size=64, epochs=10)
model.summary()


W0919 11:04:23.504770  4752 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0919 11:04:23.522770  4752 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0919 11:04:23.523771  4752 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0919 11:04:23.558773  4752 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.comp

acc_inputs.shape,gyro_inputs.shape  (119080, 20, 60, 1) (119080, 20, 60, 1)


W0919 11:04:25.124862  4752 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0919 11:04:25.222868  4752 deprecation.py:506] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


acc_conv3_shape [None, 20, 4, 64]
acc_conv_out.shape  (?, 20, 1, 4, 64)
gyro_conv3.shape  [None, 20, 4, 64]
gyro_conv_out.shape  (?, 20, 1, 4, 64)
sensor_conv_in  (?, 20, 2, 4, 64)
sensor_conv_in  Tensor("dropout_5/cond/Merge:0", shape=(?, 20, 2, 4, 64), dtype=float32)
sensor_conv3.shape  [None, 20, 2, 4, 64]
sensor_conv_out.shape  (?, 20, 2, 4, 64)
shape before gru  (?, 20, ?)
gru1.shape  (?, ?, 120)


W0919 11:04:27.193980  4752 deprecation_wrapper.py:119] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



gru2.shape  (?, ?, 120)
output of GRU  (?, 120)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input1 (InputLayer)             (None, 20, 60, 1)    0                                            
__________________________________________________________________________________________________
input2 (InputLayer)             (None, 20, 60, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 20, 8, 64)    1216        input1[0][0]                     
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 20, 8, 64)    1216        input2[0][0]                     
_____________________________________________________________

In [5]:
# adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.5, beta_2=0.9, epsilon=None, decay=0.0, amsgrad=False)

# model = Model(inputs=[input1,input2], outputs=outputs)
# model.compile(loss='categorical_crossentropy',
#       optimizer=adam,
#       metrics=['accuracy'])
# acc,gyo = np.split(x_train,2,axis=2)
model.fit([acc,gyo], train_y, batch_size=64,validation_split=0.1, epochs=10)


W0919 11:05:37.113312  4752 deprecation.py:323] From C:\Users\AYA04_Intern\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 107172 samples, validate on 11908 samples
Epoch 1/10
107172/107172 [==============================] - 392s 4ms/step - loss: 0.5289 - acc: 0.7809 - val_loss: 0.3599 - val_acc: 0.8873
Epoch 2/10
107172/107172 [==============================] - 385s 4ms/step - loss: 0.2130 - acc: 0.9353 - val_loss: 0.3004 - val_acc: 0.9197
Epoch 3/10
107172/107172 [==============================] - 389s 4ms/step - loss: 0.1655 - acc: 0.9510 - val_loss: 0.2715 - val_acc: 0.9342
Epoch 4/10
107172/107172 [==============================] - 391s 4ms/step - loss: 0.1412 - acc: 0.9594 - val_loss: 0.2337 - val_acc: 0.9452
Epoch 5/10
107172/107172 [==============================] - 388s 4ms/step - loss: 0.1253 - acc: 0.9647 - val_loss: 0.2404 - val_acc: 0.9407
Epoch 6/10
107172/107172 [==============================] - 383s 4ms/step - loss: 0.1125 - acc: 0.9690 - val_loss: 0.2162 - val_acc: 0.9490
Epoch 7/10
107172/107172 [==============================] - 384s 4ms/step - loss: 0.1058 - acc: 0.9714 - val_

In [6]:
from keras.models import load_model
 
model.save('DeepSense_keras.h5')  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model
 
# returns a compiled model
# identical to the previous one
#model = load_model('DeepSense_keras_nomerge.h5')

In [7]:
# adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.5, beta_2=0.9, epsilon=None, decay=0.0, amsgrad=False)
# model.compile(loss='categorical_crossentropy',
#       optimizer=adam,
#       metrics=['accuracy'])

# model.fit([acc,gyo], train_y, batch_size=64,validation_split=0.2, epochs=10)

In [8]:
test_x = np.expand_dims(eval_x, axis=3)
acc_test,gyo_test = np.split(test_x,2,axis=2)
test_y = eval_y
test_x = [acc_test,gyo_test]
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn import metrics

# print ( "Loss = " + str( preds[0] ) )
# print ( "Test Accuracy = " + str( preds[1] ) )
print(model.metrics_names)
loss, acc = model.evaluate(test_x,test_y, batch_size=64)
print('loss, acc ',loss,acc)


y_pred = model.predict(test_x, batch_size=128, verbose=1)
# y_pred_bool = np.argmax(preds, axis=1)
y_pred = (y_pred > 0.5)

for i in range(len(y_pred)):
        max_value=max(y_pred[i])
        for j in range(len(y_pred[i])):
            if max_value==y_pred[i][j]:
                y_pred[i][j]=1
            else:
                y_pred[i][j]=0

print(classification_report(test_y, y_pred))
print(metrics.f1_score(test_y, y_pred, average="weighted"))
print(confusion_matrix(test_y.argmax(axis=1), y_pred.argmax(axis=1)))

['loss', 'acc']
1193/1193 [==============================] - 1s 1ms/step
loss, acc  0.33391250543802203 0.9161777031691456
1193/1193 [==============================] - 1s 1ms/step
              precision    recall  f1-score   support

           0       0.94      0.95      0.95       118
           1       0.00      0.00      0.00         4
           2       0.94      1.00      0.97       231
           3       0.96      0.86      0.91       111
           4       0.82      0.99      0.89       360
           5       0.98      0.82      0.89       369

   micro avg       0.90      0.92      0.91      1193
   macro avg       0.77      0.77      0.77      1193
weighted avg       0.91      0.92      0.91      1193
 samples avg       0.92      0.92      0.92      1193

0.9116311006363353
[[112   0   6   0   0   0]
 [  0   0   4   0   0   0]
 [  1   0 230   0   0   0]
 [  1   0   0  96  12   2]
 [  0   0   0   0 356   4]
 [  5   0   0   0  64 300]]


In [9]:
# logits = deepSense(train_x, True, name='deepSense')

# predict = tf.argmax(logits, axis=1)

# batchLoss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=train_y)
# loss = tf.reduce_mean(batchLoss)

# with tf.Session() as sess:
# 	tf.global_variables_initializer().run()
# 	# coord = tf.train.Coordinator()
# 	# threads = tf.train.start_queue_runners(coord=coord)

# 	for iteration in range(TOTAL_ITER_NUM):

# 		# _, lossV, _trainY, _predict = sess.run([discOptimizer, loss, trainY, predict], feed_dict = {
# 		# 	train_status: True
# 		# 	})
# 		_, lossV, _trainY, _predict = sess.run([discOptimizer, loss, train_y, predict])
# 		_label = np.argmax(_trainY, axis=1)
# 		_accuracy = np.mean(_label == _predict)

In [19]:
import pandas as pd
df = pd.DataFrame(train_y)
for ele in range(6):
    print('category ',ele,  dict(df[ele].value_counts()))
df1 = pd.DataFrame(eval_y)
print()
for ele in range(6):
    print('category ',ele,  dict(df1[ele].value_counts()))

category  0 {0.0: 105040, 1.0: 14040}
category  1 {0.0: 108870, 1.0: 10210}
category  2 {0.0: 106900, 1.0: 12180}
category  3 {0.0: 111250, 1.0: 7830}
category  4 {0.0: 80310, 1.0: 38770}
category  5 {0.0: 83030, 1.0: 36050}

category  0 {0.0: 1075, 1.0: 118}
category  1 {0.0: 1189, 1.0: 4}
category  2 {0.0: 962, 1.0: 231}
category  3 {0.0: 1082, 1.0: 111}
category  4 {0.0: 833, 1.0: 360}
category  5 {0.0: 824, 1.0: 369}
